# Notebook 01 - Análise Exploratória de Dados (EDA)

**Projeto:** Previsão de Churn em Telecomunicações  
**Autores:** Pedro Dias, Gustavo Rodrigues  
**Data:** Dezembro 2025

---

## Objetivo

Realizar uma análise exploratória completa do dataset de churn para:
1. Entender a estrutura e qualidade dos dados
2. Identificar padrões e correlações
3. Gerar insights para seleção de features
4. Preparar os dados para modelagem

In [ ]:
# ========== SETUP ==========
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Bibliotecas carregadas com sucesso!")
print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")

## 1. Carregamento dos Dados

In [ ]:
# Carregar dataset
url = "https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv"

try:
    df = pd.read_csv(url)
    print("Dataset carregado com sucesso!")
except Exception as e:
    print(f"Erro ao carregar: {e}")
    print("Tentando URL alternativa...")
    url_alt = "https://raw.githubusercontent.com/marvin-rubia/Churn-Analysis-Prediction/main/WA_Fn-UseC_-Telco-Customer-Churn.csv"
    df = pd.read_csv(url_alt)
    print("Dataset carregado da URL alternativa!")

print(f"\nDimensões: {df.shape[0]} linhas x {df.shape[1]} colunas")

## 2. Visão Geral dos Dados

In [ ]:
# Primeiras linhas
print("=" * 80)
print("AMOSTRA DOS DADOS")
print("=" * 80)
display(df.head())

# Informações gerais
print("\n" + "=" * 80)
print("INFORMAÇÕES GERAIS")
print("=" * 80)
df.info()

In [ ]:
# Estatísticas descritivas
print("=" * 80)
print("ESTATÍSTICAS DESCRITIVAS - VARIÁVEIS NUMÉRICAS")
print("=" * 80)
display(df.describe().T)

## 3. Análise de Qualidade dos Dados

In [ ]:
# Verificar valores ausentes
print("=" * 80)
print("ANÁLISE DE VALORES AUSENTES")
print("=" * 80)

missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Valores Ausentes': missing,
    'Percentual (%)': missing_pct
})
missing_df = missing_df[missing_df['Valores Ausentes'] > 0].sort_values('Valores Ausentes', ascending=False)

if len(missing_df) > 0:
    display(missing_df)
else:
    print("Nenhum valor ausente detectado!")

# Verificar TotalCharges (problema conhecido)
print("\n" + "=" * 80)
print("VERIFICAÇÃO ESPECIAL: TotalCharges")
print("=" * 80)
print(f"Tipo da coluna: {df['TotalCharges'].dtype}")
print(f"Valores únicos que não são números:")
print(df[pd.to_numeric(df['TotalCharges'], errors='coerce').isnull()]['TotalCharges'].value_counts())

### Limpeza de Dados

In [ ]:
# Converter TotalCharges para numérico
print("Convertendo TotalCharges para numérico...")
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Contar NAs após conversão
total_nas = df['TotalCharges'].isnull().sum()
print(f"   Valores NA gerados: {total_nas}")

if total_nas > 0:
    print(f"\nAnalisando os {total_nas} registros com NA...")
    na_records = df[df['TotalCharges'].isnull()]
    print(f"   Tenure médio: {na_records['tenure'].mean():.2f} meses")
    print(f"   MonthlyCharges médio: R$ {na_records['MonthlyCharges'].mean():.2f}")
    print("\n   Decisão: Remover esses registros (representam < 0.2% dos dados)")
    
    df_clean = df.dropna(subset=['TotalCharges']).copy()
    print(f"\nDataset limpo: {df_clean.shape[0]} registros restantes")
else:
    df_clean = df.copy()
    print("Nenhuma limpeza necessária!")

# Salvar dataset limpo
# df_clean.to_csv('../data/telco_churn_clean.csv', index=False)
print("\nDataset limpo pronto para análise!")

## 4. Análise da Variável Target (Churn)

In [ ]:
# Distribuição do Churn
print("=" * 80)
print("DISTRIBUIÇÃO DA VARIÁVEL TARGET: CHURN")
print("=" * 80)

churn_counts = df_clean['Churn'].value_counts()
churn_pct = df_clean['Churn'].value_counts(normalize=True) * 100

churn_summary = pd.DataFrame({
    'Quantidade': churn_counts,
    'Percentual (%)': churn_pct.round(2)
})
display(churn_summary)

# Visualização
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de barras
churn_counts.plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'])
axes[0].set_title('Distribuição de Churn (Contagem)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Churn', fontsize=12)
axes[0].set_ylabel('Quantidade de Clientes', fontsize=12)
axes[0].set_xticklabels(['Não', 'Sim'], rotation=0)
axes[0].grid(axis='y', alpha=0.3)

# Adicionar valores nas barras
for i, v in enumerate(churn_counts):
    axes[0].text(i, v + 50, str(v), ha='center', fontweight='bold')

# Gráfico de pizza
colors = ['#2ecc71', '#e74c3c']
explode = (0.05, 0.1)
axes[1].pie(churn_pct, labels=['Não Churn', 'Churn'], autopct='%1.1f%%',
            colors=colors, explode=explode, shadow=True, startangle=90)
axes[1].set_title('Proporção de Churn', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nDataset DESBALANCEADO: {churn_pct['No']:.1f}% Não-Churn vs {churn_pct['Yes']:.1f}% Churn")
print("   Implicação: Precisamos usar stratify no train_test_split!")

## 5. Análise de Variáveis Numéricas

In [ ]:
# Identificar colunas numéricas
numeric_cols = df_clean.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_cols = [col for col in numeric_cols if col != 'customerID']

print(f"Variáveis numéricas identificadas: {numeric_cols}")
print("\n" + "=" * 80)
print("ESTATÍSTICAS POR CLASSE (CHURN)")
print("=" * 80)

for col in numeric_cols:
    print(f"\n{col}:")
    stats = df_clean.groupby('Churn')[col].agg(['mean', 'median', 'std'])
    display(stats.round(2))

In [ ]:
# Visualização: Distribuições comparativas
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, col in enumerate(numeric_cols):
    ax = axes[idx]
    
    # Boxplot
    df_clean.boxplot(column=col, by='Churn', ax=ax, patch_artist=True)
    ax.set_title(f'Distribuição de {col} por Churn', fontsize=12, fontweight='bold')
    ax.set_xlabel('Churn', fontsize=10)
    ax.set_ylabel(col, fontsize=10)
    plt.sca(ax)
    plt.xticks([1, 2], ['Não', 'Sim'])

plt.suptitle('')  # Remover título automático do pandas
plt.tight_layout()
plt.show()

### Insights Principais - Variáveis Numéricas

**tenure (Tempo como cliente):**
- Clientes que dão churn têm tenure significativamente menor
- Mediana ~10 meses para churn vs ~38 meses para não-churn
- **Insight:** Primeiros 12 meses são críticos para retenção!

**MonthlyCharges:**
- Clientes com churn pagam mensalidades mais altas
- Pode indicar insatisfação com relação custo-benefício

**TotalCharges:**
- Fortemente correlacionado com tenure
- Clientes fiéis acumulam maior valor total

## 6. Análise de Variáveis Categóricas

In [ ]:
# Identificar colunas categóricas
categorical_cols = df_clean.select_dtypes(include=['object']).columns.tolist()
categorical_cols = [col for col in categorical_cols if col not in ['customerID', 'Churn']]

print(f"Variáveis categóricas identificadas ({len(categorical_cols)}):")
for col in categorical_cols:
    print(f"  - {col}: {df_clean[col].nunique()} categorias")

In [ ]:
# Análise de cada variável categórica vs Churn
important_cats = ['Contract', 'InternetService', 'PaymentMethod', 'OnlineSecurity', 
                  'TechSupport', 'PaperlessBilling']

fig, axes = plt.subplots(3, 2, figsize=(16, 18))
axes = axes.flatten()

for idx, col in enumerate(important_cats):
    ax = axes[idx]
    
    # Calcular taxa de churn por categoria
    churn_rate = df_clean.groupby(col)['Churn'].apply(lambda x: (x == 'Yes').sum() / len(x) * 100)
    churn_rate = churn_rate.sort_values(ascending=False)
    
    # Plotar
    churn_rate.plot(kind='barh', ax=ax, color='#e74c3c')
    ax.set_title(f'Taxa de Churn por {col}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Taxa de Churn (%)', fontsize=10)
    ax.set_ylabel('')
    ax.grid(axis='x', alpha=0.3)
    
    # Adicionar valores
    for i, v in enumerate(churn_rate):
        ax.text(v + 1, i, f'{v:.1f}%', va='center')

plt.tight_layout()
plt.show()

In [ ]:
# Tabela detalhada: Contract
print("=" * 80)
print("ANÁLISE DETALHADA: TIPO DE CONTRATO")
print("=" * 80)

contract_analysis = pd.crosstab(df_clean['Contract'], df_clean['Churn'], normalize='index') * 100
contract_analysis['Total'] = df_clean['Contract'].value_counts()
contract_analysis = contract_analysis.round(2)
display(contract_analysis)

print("\nInsight: Contratos mensais têm taxa de churn 10x maior que contratos de 2 anos!")

### Insights Principais - Variáveis Categóricas

**Contract (Tipo de Contrato):** FEATURE MAIS IMPORTANTE
- Month-to-month: ~42% de churn
- One year: ~11% de churn  
- Two year: ~3% de churn
- **Ação:** Incentivar migração para contratos longos!

**InternetService:**
- Fiber optic: maior taxa de churn (~42%)
- DSL: churn moderado (~19%)
- Sem internet: baixo churn (~7%)
- **Hipótese:** Problemas de qualidade com fibra?

**PaymentMethod:**
- Electronic check: ~45% de churn (ALERTA!)
- Outros métodos: ~15-18% de churn
- **Ação:** Investigar UX do pagamento eletrônico

**OnlineSecurity & TechSupport:**
- Clientes SEM esses serviços têm ~42% de churn
- Clientes COM esses serviços têm ~15% de churn
- **Ação:** Oferecer gratuitamente nos primeiros meses

## 7. Análise de Correlações

In [ ]:
# Preparar dados para correlação (converter Churn para numérico)
df_corr = df_clean.copy()
df_corr['Churn_num'] = (df_corr['Churn'] == 'Yes').astype(int)

# Calcular correlações
numeric_for_corr = ['tenure', 'MonthlyCharges', 'TotalCharges', 'SeniorCitizen', 'Churn_num']
correlation_matrix = df_corr[numeric_for_corr].corr()

# Visualização
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='RdYlGn_r',
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlação - Variáveis Numéricas', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Correlações com Churn
print("\n" + "=" * 80)
print("CORRELAÇÕES COM CHURN")
print("=" * 80)
churn_corr = correlation_matrix['Churn_num'].drop('Churn_num').sort_values(ascending=False)
display(pd.DataFrame(churn_corr, columns=['Correlação']).round(3))

## 8. Feature Engineering - Criação de Novas Variáveis

In [ ]:
# Criar features derivadas (opcional para melhorar o modelo)
df_clean['ChargesPerMonth'] = df_clean['TotalCharges'] / (df_clean['tenure'] + 1)  # +1 para evitar divisão por 0
df_clean['TenureGroup'] = pd.cut(df_clean['tenure'], 
                                   bins=[0, 12, 24, 48, 100], 
                                   labels=['0-1 ano', '1-2 anos', '2-4 anos', '4+ anos'])

print("Novas features criadas:")
print("  - ChargesPerMonth: Gasto médio mensal")
print("  - TenureGroup: Categorização do tempo de contrato")

# Análise da nova feature
print("\n" + "=" * 80)
print("ANÁLISE: CHURN POR GRUPO DE TENURE")
print("=" * 80)
tenure_churn = pd.crosstab(df_clean['TenureGroup'], df_clean['Churn'], normalize='index') * 100
display(tenure_churn.round(2))

## 9. Seleção Final de Features

In [ ]:
# Features selecionadas para modelagem
selected_features = [
    # Numéricas
    'tenure',
    'MonthlyCharges',
    'TotalCharges',
    # Categóricas importantes
    'Contract',
    'InternetService',
    'PaymentMethod',
    'OnlineSecurity',
    'TechSupport',
    'PaperlessBilling',
    'SeniorCitizen'
]

print("=" * 80)
print("FEATURES SELECIONADAS PARA MODELAGEM")
print("=" * 80)
print(f"\nTotal: {len(selected_features)} features\n")

for i, feat in enumerate(selected_features, 1):
    dtype = 'Numérica' if df_clean[feat].dtype in ['int64', 'float64'] else 'Categórica'
    print(f"{i:2d}. {feat:20s} ({dtype})")

print("\n" + "=" * 80)
print("CRITÉRIOS DE SELEÇÃO:")
print("=" * 80)
print("Alta correlação com target (> 0.15)")
print("Relevância de negócio (acionável)")
print("Baixa multicolinearidade")
print("Disponível no momento da predição")

## 10. Preparação para Modelagem

In [ ]:
# Preparar X e y
X = df_clean[selected_features].copy()
y = df_clean['Churn'].copy()

print("=" * 80)
print("DADOS PREPARADOS PARA MODELAGEM")
print("=" * 80)
print(f"\nFormato de X: {X.shape}")
print(f"Formato de y: {y.shape}")
print(f"\nDistribuição de y:")
display(y.value_counts())

# Salvar datasets processados (opcional)
# X.to_csv('../data/X_features.csv', index=False)
# y.to_csv('../data/y_target.csv', index=False)

print("\nDados prontos para o Notebook 02 - Modelagem!")

## Resumo da EDA

### Principais Descobertas:

1. **Dataset:**
   - 7.032 clientes após limpeza
   - 21 features originais
   - Target desbalanceado (73% Não-Churn, 27% Churn)

2. **Features Mais Relevantes:**
   - **Contract:** Maior preditor de churn
   - **tenure:** Inversamente proporcional ao churn
   - **InternetService (Fiber):** Alta taxa de churn
   - **PaymentMethod (E-check):** Correlação forte com churn

3. **Insights Acionáveis:**
   - Focar retenção nos primeiros 12 meses
   - Incentivar contratos longos (descontos progressivos)
   - Investigar qualidade da fibra ótica
   - Melhorar UX do pagamento eletrônico
   - Oferecer serviços de suporte gratuitamente no início

4. **Preparação Concluída:**
   - 10 features selecionadas
   - Dados limpos e validados
   - Pronto para encoding e modelagem

---

**Próximo passo:** Notebook 02 - Modelagem Comparativa